# ProtoNN in Tensorflow

This is a simple notebook that illustrates the usage of Tensorflow implementation of ProtoNN. We are using the USPS dataset. Please refer to `fetch_usps.py` and `process_usps.py`for more details on downloading the dataset.

In [1]:
# Copyright (c) Microsoft Corporation. All rights reserved.
# Licensed under the MIT license.

from __future__ import print_function
import sys
import os
import numpy as np
import tensorflow as tf

sys.path.insert(0, '../../')
from edgeml.trainer.protoNNTrainer import ProtoNNTrainer
from edgeml.graph.protoNN import ProtoNN
import edgeml.utils as utils
import helpermethods as helper

# USPS Data

It is assumed that the USPS data has already been downloaded and set up with the help of [fetch_usps.py](fetch_usps.py) and is placed in the `./usps10` subdirectory.

In [2]:
# Load data
DATA_DIR = './usps10'
x_train = np.load(DATA_DIR + '/x_train.npy')
y_train = np.load(DATA_DIR + '/y_train.npy')
x_test = np.load(DATA_DIR + '/x_test.npy')
y_test = np.load(DATA_DIR + '/y_test.npy')
dataDimension = x_train.shape[1]
numClasses = y_train.shape[1]

# Model Parameters

Note that ProtoNN is very sensitive to the value of the hyperparameter $\gamma$, here stored in valiable `GAMMA`. If `GAMMA` is set to `None`, median heuristic will be used to estimate a good value of $\gamma$ through the `helper.getGamma()` method. This method also returns the corresponding `W` and `B` matrices which should be used to initialize ProtoNN (as is done here).

In [3]:
PROJECTION_DIM = 60
NUM_PROTOTYPES = 60
REG_W = 0.000005
REG_B = 0.0
REG_Z = 0.00005
SPAR_W = 0.8
SPAR_B = 1.0
SPAR_Z = 1.0
LEARNING_RATE = 0.05
NUM_EPOCHS = 200
BATCH_SIZE = 32
GAMMA = 0.0015

In [4]:
W, B, gamma = helper.getGamma(GAMMA, PROJECTION_DIM, dataDimension,
                       NUM_PROTOTYPES, x_train)

In [5]:
# Setup input and train protoNN
X = tf.placeholder(tf.float32, [None, dataDimension], name='X')
Y = tf.placeholder(tf.float32, [None, numClasses], name='Y')
protoNN = ProtoNN(dataDimension, PROJECTION_DIM,
                  NUM_PROTOTYPES, numClasses,
                  gamma, W=W, B=B)
trainer = ProtoNNTrainer(protoNN, REG_W, REG_B, REG_Z,
                         SPAR_W, SPAR_B, SPAR_Z,
                         LEARNING_RATE, X, Y, lossType='xentropy')
sess = tf.Session()
trainer.train(BATCH_SIZE, NUM_EPOCHS, sess, x_train, x_test, y_train, y_test,
              printStep=600, valStep=10)



Epoch:   0 Batch:   0 Loss: 6.37024 Accuracy: 0.21875
Epoch:   1 Batch:   0 Loss: 1.67235 Accuracy: 0.40625
Epoch:   2 Batch:   0 Loss: 0.83359 Accuracy: 0.87500
Epoch:   3 Batch:   0 Loss: 0.59837 Accuracy: 0.90625
Epoch:   4 Batch:   0 Loss: 0.46367 Accuracy: 0.93750
Epoch:   5 Batch:   0 Loss: 0.38596 Accuracy: 0.93750
Epoch:   6 Batch:   0 Loss: 0.33876 Accuracy: 0.93750
Epoch:   7 Batch:   0 Loss: 0.30960 Accuracy: 0.93750
Epoch:   8 Batch:   0 Loss: 0.29468 Accuracy: 0.96875
Epoch:   9 Batch:   0 Loss: 0.28825 Accuracy: 0.96875
Test Loss: 0.51032 Accuracy: 0.89347
Epoch:  10 Batch:   0 Loss: 0.28584 Accuracy: 0.96875
Epoch:  11 Batch:   0 Loss: 0.28527 Accuracy: 0.96875
Epoch:  12 Batch:   0 Loss: 0.28501 Accuracy: 0.96875
Epoch:  13 Batch:   0 Loss: 0.28400 Accuracy: 0.96875
Epoch:  14 Batch:   0 Loss: 0.28342 Accuracy: 0.96875
Epoch:  15 Batch:   0 Loss: 0.28222 Accuracy: 0.96875
Epoch:  16 Batch:   0 Loss: 0.28001 Accuracy: 0.96875
Epoch:  17 Batch:   0 Loss: 0.27642 Accuracy:

Epoch: 143 Batch:   0 Loss: 0.18951 Accuracy: 0.96875
Epoch: 144 Batch:   0 Loss: 0.18934 Accuracy: 0.96875
Epoch: 145 Batch:   0 Loss: 0.18907 Accuracy: 0.96875
Epoch: 146 Batch:   0 Loss: 0.18870 Accuracy: 0.96875
Epoch: 147 Batch:   0 Loss: 0.18837 Accuracy: 0.96875
Epoch: 148 Batch:   0 Loss: 0.18806 Accuracy: 0.96875
Epoch: 149 Batch:   0 Loss: 0.18782 Accuracy: 0.96875
Test Loss: 0.42528 Accuracy: 0.91937
Epoch: 150 Batch:   0 Loss: 0.18746 Accuracy: 0.96875
Epoch: 151 Batch:   0 Loss: 0.18714 Accuracy: 0.96875
Epoch: 152 Batch:   0 Loss: 0.18685 Accuracy: 1.00000
Epoch: 153 Batch:   0 Loss: 0.18649 Accuracy: 1.00000
Epoch: 154 Batch:   0 Loss: 0.18628 Accuracy: 1.00000
Epoch: 155 Batch:   0 Loss: 0.18598 Accuracy: 1.00000
Epoch: 156 Batch:   0 Loss: 0.18584 Accuracy: 1.00000
Epoch: 157 Batch:   0 Loss: 0.18566 Accuracy: 1.00000
Epoch: 158 Batch:   0 Loss: 0.18546 Accuracy: 1.00000
Epoch: 159 Batch:   0 Loss: 0.18523 Accuracy: 1.00000
Test Loss: 0.42251 Accuracy: 0.91937
Epoch: 1

# Model Evaluation

In [6]:
acc = sess.run(protoNN.accuracy, feed_dict={X: x_test, Y: y_test})
# W, B, Z are tensorflow graph nodes
W, B, Z, _ = protoNN.getModelMatrices()
matrixList = sess.run([W, B, Z])
sparcityList = [SPAR_W, SPAR_B, SPAR_Z]
nnz, size, sparse = helper.getModelSize(matrixList, sparcityList)
print("Final test accuracy", acc)
print("Model size constraint (Bytes): ", size)
print("Number of non-zeros: ", nnz)
nnz, size, sparse = helper.getModelSize(matrixList, sparcityList, expected=False)
print("Actual model size: ", size)
print("Actual non-zeros: ", nnz)

Final test accuracy 0.922272
Model size constraint (Bytes):  78240
Number of non-zeros:  19560
Actual model size:  78240
Actual non-zeros:  16488
